In [1]:

from google.cloud import bigquery
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

client = bigquery.Client()
dataset_id = 'airport_database'

dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"


try:
  dataset = client.create_dataset(dataset)  # Make an API request.
  print("Dataset {} created.".format(dataset.dataset_id))
except Exception:
  print('Database already exsist')


Dataset airport_database created.


In [2]:

dataset_id = 'airport_database'
table_id = 'airport'

dataset_ref = client.dataset(dataset_id)

# Construct a BigQuery table object
table_ref = dataset_ref.table(table_id)
table = bigquery.Table(table_ref)

# Configure the load job.
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    skip_leading_rows=1,
    source_format=bigquery.SourceFormat.CSV,
    write_disposition="WRITE_TRUNCATE"
)


In [3]:

# Specify the Cloud Storage URI of the file to load.
uri = "gs://labs.roitraining.com/data-to-ai-workshop/airports.csv"

# Start the load job.
load_job = client.load_table_from_uri(
    uri, table, job_config=job_config
)

load_job.result()

LoadJob<project=qwiklabs-gcp-03-e694fe7b6067, location=US, id=8b498cef-532b-4e6c-880c-27097aeff480>

In [2]:
query_1 = """SELECT * FROM `airport_database.airport`"""

airports_df = client.query(query_1).to_dataframe()
airports_df.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total RF Heliport,40.070985,-74.933689,11,NA,US,US-PA,Bensalem,False,None,None,K00A,00A,https://www.penndot.pa.gov/TravelInPA/airports...,None,None
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435,NA,US,US-KS,Leoti,False,None,None,00AA,00AA,None,None,None
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450,NA,US,US-AK,Anchor Point,False,None,None,00AK,00AK,None,None,None
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820,NA,US,US-AL,Harvest,False,None,None,00AL,00AL,None,None,None
4,506791,00AN,small_airport,Katmai Lodge Airport,59.093287,-156.456699,80,NA,US,US-AK,King Salmon,False,None,None,00AN,00AN,None,None,None


In [5]:
airports_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82893 entries, 0 to 82892
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 82893 non-null  Int64  
 1   ident              82893 non-null  object 
 2   type               82893 non-null  object 
 3   name               82893 non-null  object 
 4   latitude_deg       82893 non-null  float64
 5   longitude_deg      82893 non-null  float64
 6   elevation_ft       68425 non-null  Int64  
 7   continent          82893 non-null  object 
 8   iso_country        82893 non-null  object 
 9   iso_region         82893 non-null  object 
 10  municipality       78207 non-null  object 
 11  scheduled_service  82893 non-null  boolean
 12  icao_code          7771 non-null   object 
 13  iata_code          9095 non-null   object 
 14  gps_code           43217 non-null  object 
 15  local_code         35742 non-null  object 
 16  home_link          427

In [6]:
airports_df.describe()

,id,latitude_deg,longitude_deg,elevation_ft
count,82893.0,82893.000000,82893.000000,68425.0
mean,192913.332996,26.018664,-27.773870,1294.158845
std,189281.692759,26.137282,85.634096,1661.808328
min,2.0,-90.000000,-179.876999,-1266.0
25%,20878.0,13.265263,-93.461304,210.0
50%,44392.0,35.200500,-65.894721,728.0
75%,341236.0,43.070974,23.472500,1601.0
max,596496.0,82.750000,179.975700,17372.0


In [7]:
unique_per_col = {col: airports_df[col].unique() for col in airports_df.columns}

for col, values in unique_per_col.items():
    print(f"{col}: {values}")

id: <IntegerArray>
[  6523, 323361,   6524,   6525, 506791, 322127,   6527,   6528, 324424,
 322658,
 ...
 307013,  27243,  44182,  27244, 317861,  32753,  46378, 307326, 346788,
 313629]
Length: 82893, dtype: Int64
ident: ['00A' '00AA' '00AK' ... 'ZZ-0002' 'ZZ-0003' 'ZZZZ']
type: ['heliport' 'small_airport' 'seaplane_base' 'balloonport' 'closed'
 'medium_airport' 'large_airport']
name: ['Total RF Heliport' 'Aero B Ranch Airport' 'Lowell Field' ...
 'Glorioso Islands Airstrip' 'Fainting Goat Airport'
 'Satsuma Iōjima Airport']
latitude_deg: [ 40.070985    38.704022    59.947733   ... -11.58427778  32.110587
  30.784722  ]
longitude_deg: [ -74.933689   -101.473911   -151.692524   ...   47.29638889  -97.356312
  130.270556  ]
elevation_ft: <IntegerArray>
[   11,  3435,   450,   820,    80,  1100,  3810,  3038,    87,  3350,
 ...
  4751, 14472,  3736, 13346, 11600, 11327, 14042, 11713,  9675, 12959]
Length: 6409, dtype: Int64
continent: ['NA' 'OC' 'AF' 'AN' 'SA' 'EU' 'AS']
iso_country: ['

In [8]:
print(airports_df.isnull().sum())

id                       0
ident                    0
type                     0
name                     0
latitude_deg             0
longitude_deg            0
elevation_ft         14468
continent                0
iso_country              0
iso_region               0
municipality          4686
scheduled_service        0
icao_code            75122
iata_code            73798
gps_code             39676
local_code           47151
home_link            78619
wikipedia_link       66399
keywords             62819
dtype: int64


In [16]:
airports_df.groupby('type').size()

,0
type,
balloonport,57
closed,12493
heliport,21796
large_airport,482
medium_airport,4685
seaplane_base,1223
small_airport,42157


## **Filter only large_airports**

In [27]:
airports_df_US = airports_df[(airports_df['type'] == 'large_airport') & (airports_df['iso_country'] == 'US')]
airports_df_US.count()

,0
id,71
ident,71
type,71
name,71
latitude_deg,71
longitude_deg,71
elevation_ft,71
continent,71
iso_country,71
iso_region,71


In [20]:
# !pip install requests==2.31.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.31.0 which is incompatible.
google-adk 1.17.0 requires requests<3.0.0,>=2.32.4, but you have requests 2.31.0 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.33.1 which is incompatible.
datasets 4.0.0 requires requests>=2.32.2, but you have requests 2.31.0 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, 

## **Create or replace Gemini model**

In [13]:
%%bigquery

try:
CREATE OR REPLACE MODEL `airport_database.gemini_airport_model`
REMOTE WITH CONNECTION DEFAULT
OPTIONS(ENDPOINT = 'gemini-2.0-flash-001');

Query is running:   0%|          |

""


## **Fetch weather and stage data**

In [51]:
import requests
import json
import pandas as pd
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from datetime import datetime
import time # To respect API rate limits

# --- Configuration ---
NWS_API_BASE = "https://api.weather.gov"
YOUR_GCP_PROJECT_ID = "qwiklabs-gcp-03-e694fe7b6067" # Replace with your project ID
BIGQUERY_DATASET_ID = "airport_database"
BIGQUERY_STAGING_TABLE_ID = "nws_raw_weather_staging"

# Initialize BigQuery client
bigquery_client = bigquery.Client(project=YOUR_GCP_PROJECT_ID)

# --- Function to get NWS grid points ---
def get_nws_grid_points(latitude, longitude):
    """Fetches NWS grid points (cwa, gridX, gridY) for a given lat/lon."""
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    headers = {'User-Agent': 'GCPWeatherApp/1.0 (vrunda.patel@ttecdigital.com)'} # NWS requires a User-Agent
    try:
        response = requests.get(points_url, headers=headers, timeout=10)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        properties = data.get('properties', {})
        return {
            'cwa': properties.get('cwa'),
            'gridX': properties.get('gridX'),
            'gridY': properties.get('gridY'),
            'forecastHourlyUrl': properties.get('forecastHourly') # Often useful for current conditions
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching NWS grid points for {latitude},{longitude}: {e}")
        return None
    except json.JSONDecodeError:
        print(f"Error decoding JSON for NWS grid points for {latitude},{longitude}")
        return None

# --- Function to get current weather from NWS (using hourly forecast as a proxy) ---
def get_current_weather(grid_data):
    """Fetches current weather conditions using the hourly forecast endpoint."""
    if not grid_data or not grid_data.get('forecastHourlyUrl'):
        return None

    hourly_forecast_url = grid_data['forecastHourlyUrl']
    headers = {'User-Agent': 'GCPWeatherApp/1.0 (your-email@example.com)'}
    try:
        response = requests.get(hourly_forecast_url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        periods = data.get('properties', {}).get('periods', [])
        if periods:
            current_period = periods[0] # Get the most current hourly forecast
            return {
                'temperature_f': current_period.get('temperature'),
                'wind_speed_mph': int(current_period.get('windSpeed', '0 mph').split(' ')[0]), # Extract number
                'precipitation_in': 0.0, # NWS API often doesn't give direct 'precipitation_in' for current. You might need to infer or use other APIs.
                'barometric_pressure_inHg': None, # NWS API typically doesn't provide this directly in forecast.
                'humidity_percent': current_period.get('relativeHumidity', {}).get('value'),
                'weather_condition': current_period.get('shortForecast'),
                'detailed_forecast': current_period.get('detailedForecast')
            }
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching current weather from NWS for {hourly_forecast_url}: {e}")
        return None
    except json.JSONDecodeError:
        print(f"Error decoding JSON for current weather from {hourly_forecast_url}")
        return None

# --- Main script logic ---
def fetch_and_load_weather_data(airports_df_US: pd.DataFrame):
  """
    Fetches weather data, creates the destination table if it doesn't exist,
    truncates it if it does, and loads the new data.
  """
  rows_to_insert = []

  for index, row in airports_df_US.iterrows():
      airport_ident = row['ident']
      latitude = row['latitude_deg']
      longitude = row['longitude_deg']
      city = row.get('municipality', 'Unknown City') # Assuming municipality is city
      state = row.get('iso_region', '').split('-')[-1] # Extract state from iso_region (e.g., US-CA -> CA)
      country = row.get('iso_country')

      # print(f"Processing airport: {airport_ident} ({city}, {state}, {country})")

      grid_data = get_nws_grid_points(latitude, longitude)
      if grid_data:
          weather_data = get_current_weather(grid_data)
          if weather_data:
              rows_to_insert.append({
                  "airport_ident": airport_ident,
                  "city": city,
                  "state": state,
                  "country": country,
                  "date": datetime.now().strftime('%Y-%m-%d'), # Current date
                  "temperature_f": weather_data['temperature_f'],
                  "wind_speed_mph": weather_data['wind_speed_mph'],
                  "precipitation_in": weather_data['precipitation_in'],
                  "barometric_pressure_inHg": weather_data['barometric_pressure_inHg'],
                  "humidity_percent": weather_data['humidity_percent'],
                  "weather_condition": weather_data['weather_condition'],
                  "detailed_forecast": weather_data['detailed_forecast'],
                  "ingestion_timestamp": datetime.utcnow().isoformat()
              })
          else:
              print(f"Could not retrieve current weather for {airport_ident}")
      else:
          print(f"Could not retrieve NWS grid points for {airport_ident}")

      time.sleep(0.5) # Be kind to the NWS API, they have rate limits (typically 50 requests/min)

  # --- Crucial Check: Is rows_to_insert actually populated? ---
  if not rows_to_insert:
      print("\nERROR: No weather data was successfully fetched for any airport. Halting execution. "
              "Check NWS API calls and data parsing in `get_nws_grid_points` and `get_current_weather`.")
      return
  else:
      print(f"\nSuccessfully collected {len(rows_to_insert)} rows of weather data for insertion.")

  table_id = f"{YOUR_GCP_PROJECT_ID}.{BIGQUERY_DATASET_ID}.{BIGQUERY_STAGING_TABLE_ID}"
  table_ref = bigquery_client.dataset(BIGQUERY_DATASET_ID).table(BIGQUERY_STAGING_TABLE_ID)

  schema = [
        bigquery.SchemaField("airport_ident", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("city", "STRING"),
        bigquery.SchemaField("state", "STRING"),
        bigquery.SchemaField("country", "STRING"),
        bigquery.SchemaField("date", "DATE"),
        bigquery.SchemaField("temperature_f", "INTEGER"),
        bigquery.SchemaField("wind_speed_mph", "INTEGER"),
        bigquery.SchemaField("precipitation_in", "FLOAT"),
        bigquery.SchemaField("barometric_pressure_inHg", "FLOAT"),
        bigquery.SchemaField("humidity_percent", "INTEGER"),
        bigquery.SchemaField("weather_condition", "STRING"),
        bigquery.SchemaField("detailed_forecast", "STRING"),
        bigquery.SchemaField("ingestion_timestamp", "TIMESTAMP"),
    ]

  # --- Logic to Create Table if it Doesn't Exist ---
  try:
    # Check if table exists, truncate it before loading new data
    bigquery_client.get_table(table_ref)
    print(f"Table {table_id} exists. Truncating existing data...")

    truncate_query = f"TRUNCATE TABLE `{table_id}`"
    query_job = bigquery_client.query(truncate_query)
    query_job.result() # Wait for completion
    print("Table truncated.")

  except NotFound:
    print(f"Table {table_id} not found. Creating it now...")
    table = bigquery.Table(table_ref, schema= schema)
    bigquery_client.create_table(table) # Create the table
    print(f"Table {table_id} created successfully.")
    # --- RE-FETCH THE TABLE REFERENCE AFTER CREATION ---
    # This ensures the 'table_ref' object is fully initialized with the newly created table
    table_ref = bigquery_client.get_table(table_ref)
    print("Table reference re-fetched after creation.")

  except Exception as e:
      print(f"ERROR: An error occurred during table existence check or truncation: {e}")
      # If truncation failed, it's safer not to attempt insertion
      return

  # --- Load Data ---
  print(f"Attempting to load {len(rows_to_insert)} new rows into {table_id}...")
  try:
      # Use a retry mechanism for transient BigQuery API errors
      errors = bigquery_client.insert_rows_json(table_ref, rows_to_insert, retry=bigquery.DEFAULT_RETRY)
      if not errors:
          print(f"SUCCESS: {len(rows_to_insert)} rows loaded into {table_id}.")
      else:
          print(f"ERROR: Encountered errors while inserting rows into {table_id}:")
          for error in errors:
              print(f"  {error}")
          print("Please check the data types and schema for potential mismatches.")

  except Exception as e:
      print(f"CRITICAL ERROR: Failed to insert data into BigQuery: {e}")



## **Creating a fetching table**

In [52]:

fetch_and_load_weather_data(airports_df_US)

/tmp/ipython-input-2452426842.py:108: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ingestion_timestamp": datetime.utcnow().isoformat()



Successfully collected 71 rows of weather data for insertion.
Table qwiklabs-gcp-03-e694fe7b6067.airport_database.nws_raw_weather_staging exists. Truncating existing data...
Table truncated.
Attempting to load 71 new rows into qwiklabs-gcp-03-e694fe7b6067.airport_database.nws_raw_weather_staging...
SUCCESS: 71 rows loaded into qwiklabs-gcp-03-e694fe7b6067.airport_database.nws_raw_weather_staging.


In [46]:
%%bigquery

# select * from airport_database.nws_raw_weather_staging limit 10;


Query is running:   0%|          |

Downloading:   0%|          |

,airport_ident,city,state,country,date,temperature_f,wind_speed_mph,precipitation_in,barometric_pressure_inHg,humidity_percent,weather_condition,detailed_forecast,ingestion_timestamp
0,KABQ,Albuquerque,NM,US,2026-01-16,42,5,0.0,NaN,49,Sunny,,2026-01-16 17:58:22.930541+00:00
1,KADW,Camp Springs,MD,US,2026-01-16,32,9,0.0,NaN,37,Partly Sunny,,2026-01-16 17:58:23.780066+00:00
2,KATL,Atlanta,GA,US,2026-01-16,43,10,0.0,NaN,30,Sunny,,2026-01-16 17:58:24.631057+00:00
3,KAUS,Austin,TX,US,2026-01-16,65,15,0.0,NaN,43,Sunny,,2026-01-16 17:58:25.483042+00:00
4,KBDL,Hartford,CT,US,2026-01-16,28,17,0.0,NaN,33,Sunny,,2026-01-16 17:58:26.368801+00:00
5,KBNA,Nashville,TN,US,2026-01-16,44,15,0.0,NaN,38,Sunny,,2026-01-16 17:58:27.172798+00:00
6,KBOS,Boston,MA,US,2026-01-16,29,24,0.0,NaN,34,Sunny,,2026-01-16 17:58:28.019819+00:00
7,KBUF,Buffalo,NY,US,2026-01-16,23,13,0.0,NaN,65,Chance Snow Showers,,2026-01-16 17:58:28.868372+00:00
8,KBWI,Baltimore,MD,US,2026-01-16,32,9,0.0,NaN,41,Partly Sunny,,2026-01-16 17:58:29.712962+00:00
9,KCLE,Cleveland,OH,US,2026-01-16,26,15,0.0,NaN,70,Chance Snow Showers,,2026-01-16 17:58:30.568694+00:00


## **Generate prompt using Gemini**

In [47]:
%%bigquery

CREATE OR REPLACE TABLE
`airport_database.airport_weather_reports` AS (
SELECT
  ml_generate_text_llm_result AS airport_weather,
  * EXCEPT (ml_generate_text_llm_result)
FROM ML.GENERATE_TEXT(
  MODEL `airport_database.gemini_airport_model`, -- Ensure this model exists and is configured for Gemini
  (
    SELECT
      CONCAT(
        'Based on the following weather data, create a well-structured weather report or warning if conditions are severe. Focus on key details and potential impacts. If conditions are mild, state that clearly. Keep it concise, around 2-3 sentences:',
        'Location: ', city, ', ', state,', ', country,
        'Date: ', CAST(date AS STRING),
        'Temperature: ', CAST(temperature_f AS STRING), ' degrees Fahrenheit',
        'Wind speed: ', CAST(wind_speed_mph AS STRING), ' mph',
        'Precipitation: ', CAST(precipitation_in AS STRING), ' inches',
        'Humidity: ', CAST(humidity_percent AS STRING), ' percent',
        'Weather condition: ', weather_condition,
        IF(detailed_forecast IS NOT NULL, CONCAT(' (Detailed forecast: ', detailed_forecast, ')'), '')
      ) AS prompt,
      airport_ident,
      city,
      state,
      country,
      date,
      temperature_f,
      wind_speed_mph,
      precipitation_in,
      barometric_pressure_inHg,
      humidity_percent,
      weather_condition,
      detailed_forecast,
      ingestion_timestamp
    FROM
      `qwiklabs-gcp-03-e694fe7b6067.airport_database.nws_raw_weather_staging` -- Use your project ID and table name
    WHERE
      -- Optionally filter for recent data or specific airports
      ingestion_timestamp >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 HOUR)
  ),
  STRUCT(
    0.2 AS TEMPERATURE,
    256 AS MAX_OUTPUT_TOKENS,
    TRUE AS flatten_json_output
  )
))


Query is running:   0%|          |

""


In [ ]:
from datetime import datetime
import pytz # For timezones

def your_scheduled_function():
    # Your main scheduled task logic goes here
    print("Scheduled task is running...")
    # Simulate some work
    import time
    time.sleep(5)
    print("Scheduled task finished its work.")

    # Get current time
    current_time_utc = datetime.now(pytz.utc)
    current_time_local = datetime.now()
    current_time_nyc = datetime.now(pytz.timezone('America/New_York'))


    print(f"Run finished at (UTC): {current_time_utc.isoformat()}")
    print(f"Run finished at (Local/System Time): {current_time_local.isoformat()}")
    print(f"Run finished at (New York Time): {current_time_nyc.isoformat()}")

print('time schedule function created.')

In [57]:
if __name__ == "__main__":
  your_scheduled_function()

Scheduled task is running...
Scheduled task finished its work.
Run finished at (UTC): 2026-01-16T20:28:58.606609+00:00
Run finished at (Local/System Time): 2026-01-16T20:28:58.606634
Run finished at (New York Time): 2026-01-16T15:28:58.606664-05:00


In [3]:
#Just trail code


# import requests

# url = "https://api.weather.gov/"

# # Make a GET request to the URL
# response = requests.get(url)

# # Check if the request was successful (status code 200)
# if response.status_code == 200:
#     print("Content retrieved successfully. First 500 characters:")
#     # To see the output, run the code.
#     print(response.text[:500]) # Print the first 500 characters of the content
# else:
#     print(f"Failed to retrieve content. Status code: {response.status_code}")


Content retrieved successfully. First 500 characters:
<!DOCTYPE html>
<html>
    <head>
                    <title>api.weather.gov</title>
                                <link rel="stylesheet" href="/build/app.addd834c.css">
            
     <link rel="stylesheet" href="/build/swagger-ui/swagger-ui.ef52a1d9.css" />

                                <script src="/build/runtime.38e076bf.js"></script><script src="/build/app.48df84fd.js"></script>
            
    <script src="/build/swagger-ui/swagger-ui-bundle.ac392a99.js"></script>
            </he
